In [21]:
import os
from functools import partial
from multiprocessing import Pool
from typing import Optional

import cv2
import numpy as np
import pandas as pd
from pytorch_toolbelt.utils import fs
from pytorch_toolbelt.utils.distributed import is_main_process
from torch import nn
from tqdm import tqdm

from xview3.centernet.visualization import create_false_color_composite, vis_detections_opencv
from xview3.constants import PIX_TO_M
from xview3.dataset import read_multichannel_image, SigmoidNormalization, XView3DataModule
from xview3.inference import predict_multilabel_scenes
from xview3.metric import official_metric_scoring, official_metric_scoring_per_scene
from xview3.evaluation import *

In [22]:
multi_score_valid_predictions = pd.read_csv('submissions/light10/eval_public_full_light10_step_1536_tta_None/eval_public_full_light10unfiltered_predictions_step_1536_tta_None.csv')
thresholds_summary = pd.read_csv('submissions/light10/eval_public_full_light10_step_1536_tta_None/eval_public_full_light10thresholds_summary_step_1536_tta_None.csv')
scores_per_scene = pd.read_csv('submissions/light10/eval_public_full_light10_step_1536_tta_None/eval_public_full_light10scores_per_scene_step_1536_tta_None.csv')

In [23]:
# print(multi_score_valid_predictions)
print(thresholds_summary.columns)
print(scores_per_scene.columns)
print(multi_score_valid_predictions.columns)


Index(['loc_fscore', 'loc_fscore_shore', 'vessel_fscore', 'fishing_fscore',
       'length_acc', 'aggregate', 'is_vessel_threshold',
       'is_fishing_threshold', 'objectness_threshold'],
      dtype='object')
Index(['scene_id', 'global_thresholds_score', 'global_is_vessel_threshold',
       'global_is_fishing_threshold', 'local_thresholds_score',
       'local_is_vessel_threshold', 'local_is_fishing_threshold'],
      dtype='object')
Index(['vessel_length_m', 'detect_scene_row', 'detect_scene_column',
       'scene_id', 'objectness_p', 'is_vessel_p', 'is_fishing_p',
       'objectness_threshold'],
      dtype='object')


In [24]:
objectness_threshold = thresholds_summary.loc[thresholds_summary["aggregate"].idxmax(), "objectness_threshold"]
vessel_threshold = thresholds_summary.loc[thresholds_summary["aggregate"].idxmax(), "is_vessel_threshold"]
fishing_threshold = thresholds_summary.loc[thresholds_summary["aggregate"].idxmax(), "is_fishing_threshold"]
predictions = apply_thresholds(multi_score_valid_predictions, objectness_threshold, vessel_threshold, fishing_threshold)


In [26]:
predictions.to_csv('submissions/light10/eval_public_full_light10_step_1536_tta_None/final_predictions_light10.csv')